### Импорт библиотек для работы с бд и дф

In [139]:
import psycopg2
import pandas as pd

### Кредиты для коннекта к бд

In [140]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="postgres", 
                            user="admin", password="root")

### Создаем таблицу fact_orders - фактовая сущность заказов клиентов
#### order_id - Идентификатор заказа, не может быть пустым, первичный ключ, часть состовного ключа, кардинальность - низкая 
#### amount - Сумма заказа, не может быть пустым, сумма заказа должна быть > 0, кардинальность - высокая 
#### status_id - Идентификатор статусу заказа, не может быть пустым, кардинальность - низкая 
#### order_dttm - Дата заказа, не может быть пустым, кардинальность - высокая 
#### start_dttm - Дата актуальности записи, не может быть пустым, первичный ключ, часть состовного ключа, кардинальность - высокая 

In [141]:
cur = conn.cursor()
cur.execute('''
            DROP TABLE IF EXISTS fact_orders 
            ''')

cur.execute('''
           CREATE TABLE fact_orders (
           order_id INTEGER NOT NULL,
           amount NUMERIC(100,10) CHECK (amount > 0) NOT NULL,
           status_id VARCHAR(20) NOT NULL,
           order_dttm DATE NOT NULL,
           start_dttm DATE NOT NULL,
           PRIMARY KEY(order_id, start_dttm));
            ''')

### Наполняем таблицу fact_orders данными

In [142]:
cur.execute('''
            INSERT INTO fact_orders
            (order_id, amount, status_id, order_dttm, start_dttm)
            values
            (111, 5500.5, 'confirmed', '2022-09-25', '2022-09-25'),
            (111, 5500.5, 'delivered', '2022-09-25', '2022-09-26'),
            (111, 5500.5, 'canceled', '2022-09-25', '3000-12-31'),

            (222, 15500.5, 'confirmed', '2022-09-25', '2022-09-27'),
            (222, 15500.5, 'delivered', '2022-09-25', '3000-12-31'),

            (333, 125500.5, 'confirmed', '2022-09-28', '2022-09-28'),
            (333, 125500.5, 'delivered', '2022-09-28', '3000-12-31'),

            (444, 25500.5, 'confirmed', '2022-09-29', '2022-09-29'),
            (444, 25500.5, 'delivered', '2022-09-29', '2022-09-30'),
            (444, 25500.5, 'canceled', '2022-09-29', '3000-12-31');
            ''')

### Select из таблицы fact_orders

In [143]:
cur.execute('''
            select * from fact_orders
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['order_id', 'amount', 'status_id', 'order_dttm', 'start_dttm'])

,order_id,amount,status_id,order_dttm,start_dttm
0,111,5500.5000000000,confirmed,2022-09-25,2022-09-25
1,111,5500.5000000000,delivered,2022-09-25,2022-09-26
2,111,5500.5000000000,canceled,2022-09-25,3000-12-31
3,222,15500.5000000000,confirmed,2022-09-25,2022-09-27
4,222,15500.5000000000,delivered,2022-09-25,3000-12-31
5,333,125500.5000000000,confirmed,2022-09-28,2022-09-28
6,333,125500.5000000000,delivered,2022-09-28,3000-12-31
7,444,25500.5000000000,confirmed,2022-09-29,2022-09-29
8,444,25500.5000000000,delivered,2022-09-29,2022-09-30
9,444,25500.5000000000,canceled,2022-09-29,3000-12-31


### Аналитический запрос:
#### Показать id заказов и сумму заказов начиная с 2022-09-25 имеющие статус delivered

In [144]:
cur.execute('''
            select 
            order_id,
            amount
            from fact_orders
            where amount >= 10000
            and status_id ilike ('delivered')
            and order_dttm >= '2022-09-25'
            and start_dttm >= now();
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['order_id', 'amount'])

,order_id,amount
0,222,15500.5000000000
1,333,125500.5000000000


### Обновление статистики

In [145]:
cur.execute('''
            analyze fact_orders
            ''')

### Смотрим время выполнения запроса

In [149]:
cur.execute('''
            explain analyze
            select 
            order_id,
            amount
            from fact_orders
            where amount >= 10000
            and status_id ilike ('delivered')
            and order_dttm >= '2022-09-25'
            and start_dttm >= now();
            ''')
cur.fetchall()

[('Seq Scan on fact_orders  (cost=0.00..1.23 rows=1 width=12) (actual time=0.040..0.051 rows=2 loops=1)',),
 ("  Filter: ((amount >= '10000'::numeric) AND ((status_id)::text ~~* 'delivered'::text) AND (order_dttm >= '2022-09-25'::date) AND (start_dttm >= now()))",),
 ('  Rows Removed by Filter: 8',),
 ('Planning Time: 0.312 ms',),
 ('Execution Time: 0.904 ms',)]

###  Создаем индексы для таблицы fact_orders

In [150]:
cur.execute('''
            CREATE INDEX idx_amount_order_dttm
            ON fact_orders(amount, order_dttm);
            ''')

### Обновление статистики

In [151]:
cur.execute('''
            analyze fact_orders
            ''')

### Смотрим время выполнения запроса с учетом индексов

In [156]:
cur.execute('''
            explain analyze
            select 
            order_id,
            amount
            from fact_orders
            where amount >= 10000
            and status_id ilike ('delivered')
            and order_dttm >= '2022-09-25'
            and start_dttm >= now();
            ''')
cur.fetchall()

[('Seq Scan on fact_orders  (cost=0.00..1.23 rows=1 width=12) (actual time=0.020..0.027 rows=2 loops=1)',),
 ("  Filter: ((amount >= '10000'::numeric) AND ((status_id)::text ~~* 'delivered'::text) AND (order_dttm >= '2022-09-25'::date) AND (start_dttm >= now()))",),
 ('  Rows Removed by Filter: 8',),
 ('Planning Time: 0.142 ms',),
 ('Execution Time: 0.048 ms',)]

### Создаем последовательность для сущности dim_src

In [157]:
cur.execute('''
            CREATE SEQUENCE dim_src_sequence;
            ''')

### Создаем таблицу  dim_src - справочник источников
#### src_id - Идентификатор источника, не может быть пустым, первичный ключ, заполняется числовой последовательностью, кардинальность - низкая 
#### src_name - Наименование источника, не может быть пустым, кардинальность - низкая 

In [158]:
cur.execute('''
            DROP TABLE IF EXISTS dim_src 
            ''')
cur.execute('''
            CREATE TABLE dim_src (
            src_id INTEGER NOT NULL DEFAULT nextval('dim_src_sequence'),
            src_name VARCHAR(20),
            PRIMARY KEY(src_id));
            ''')

### Наполняем таблицу dim_src данными

In [159]:
cur.execute('''
            insert into dim_src (src_name)
            values
            ('VK'),
            ('INSTAGRAM'),
            ('GOOGLE'),
            ('YANDEX');
            ''')

### Select из таблицы dim_src

In [160]:
cur.execute('''
            select* from dim_src;
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['src_id', 'src_name'])

,src_id,src_name
0,1,VK
1,2,INSTAGRAM
2,3,GOOGLE
3,4,YANDEX


### Создаем таблицу fact_sessions - фактовая сущность сессий пользователей
#### session_id - Идентификатор сессии, не может быть пустым, первичный ключ, кардинальность - высокая
#### src_id -  Идентификатор источника с которого пользователь перешел, не может быть пустым, гарантирует ссылочную целостность, ссылается на справочник dim_src, кардинальность - низкая
#### user_id - Идентификатор пользователя, не может быть пустым, кардинальность - высокая
#### purchase_flg - флаг покупки, не может быть пустым, кардинальность - низкая
#### visit_dttm - Дата посещения сайта, не может быть пустым, кардинальность - высокая

In [161]:
cur.execute('''
            DROP TABLE IF EXISTS fact_sessions 
            ''')
cur.execute('''
            
            CREATE TABLE fact_sessions (
            session_id INTEGER NOT NULL,
            src_id INTEGER NOT NULL REFERENCES dim_src ON DELETE CASCADE,
            user_id INTEGER NOT NULL,
            purchase_flg INTEGER NOT NULL,
            visit_dttm DATE NOT NULL,
            PRIMARY KEY(session_id));
            ''')

### Наполняем таблицу fact_sessions данными

In [162]:
cur.execute('''
            insert into fact_sessions 
            (session_id, src_id, user_id, purchase_flg, visit_dttm)
             values
             (12311, 1, 4334, 1, '2022-09-29'),
             (12233, 2, 1114, 0, '2022-09-22'),
             (42331, 1, 4314, 1, '2022-08-29'),
             (42321, 1, 4314, 0, '2022-01-22'),
             (92321, 1, 4314, 1, '2023-01-22'),
             (72331, 3, 1514, 1, '2022-02-21'),
             (92331, 4, 6514, 1, '2022-04-22'),
             (22331, 4, 7514, 1, '2022-04-22'),
             (22321, 4, 7514, 1, '2021-04-22');
            ''')

### Select из таблицы fact_sessions

In [163]:
cur.execute('''
            select* from  fact_sessions;
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['session_id', 'src_id', 'user_id', 'purchase_flg', 'visit_dttm'])

,session_id,src_id,user_id,purchase_flg,visit_dttm
0,12311,1,4334,1,2022-09-29
1,12233,2,1114,0,2022-09-22
2,42331,1,4314,1,2022-08-29
3,42321,1,4314,0,2022-01-22
4,92321,1,4314,1,2023-01-22
5,72331,3,1514,1,2022-02-21
6,92331,4,6514,1,2022-04-22
7,22331,4,7514,1,2022-04-22
8,22321,4,7514,1,2021-04-22


### Аналитический запрос:
#### Для каждого источника посчитать количество уникальных пользователей и количество сессий, в которых была совершена покупка начиная с 2022-01-01

In [173]:
cur.execute('''
            select 
            d.src_name,
            count(distinct f.user_id) as count_users,
            count(f.session_id) as count_sessions
            from fact_sessions f
            left join dim_src d
            on d.src_id = f.src_id
            where purchase_flg = 1
            and visit_dttm >= '2022-01-01' 
            group by src_name
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['src_name', 'count_users', 'count_sessions'])

,src_name,count_users,count_sessions
0,GOOGLE,1,1
1,VK,2,3
2,YANDEX,2,2


### Обновление статистики

In [164]:
cur.execute('''
            analyze fact_sessions
            ''')
cur.execute('''
            analyze dim_src
            ''')

### Смотрим время выполнения запроса 

In [165]:
cur.execute('''
            explain analyze
            select 
            d.src_name,
            count(distinct f.user_id) as count_users,
            count(f.session_id) as count_sessions
            from fact_sessions f
            left join dim_src d
            on d.src_id = f.src_id
            where purchase_flg = 1
            and visit_dttm >= '2022-01-01' 
            group by src_name
            ''')
cur.fetchall()

[('GroupAggregate  (cost=2.36..2.47 rows=4 width=22) (actual time=0.229..0.245 rows=3 loops=1)',),
 ('  Group Key: d.src_name',),
 ('  ->  Sort  (cost=2.36..2.38 rows=7 width=14) (actual time=0.153..0.164 rows=6 loops=1)',),
 ('        Sort Key: d.src_name',),
 ('        Sort Method: quicksort  Memory: 25kB',),
 ('        ->  Hash Left Join  (cost=1.09..2.27 rows=7 width=14) (actual time=0.109..0.123 rows=6 loops=1)',),
 ('              Hash Cond: (f.src_id = d.src_id)',),
 ('              ->  Seq Scan on fact_sessions f  (cost=0.00..1.14 rows=7 width=12) (actual time=0.011..0.013 rows=6 loops=1)',),
 ("                    Filter: ((visit_dttm >= '2022-01-01'::date) AND (purchase_flg = 1))",),
 ('                    Rows Removed by Filter: 3',),
 ('              ->  Hash  (cost=1.04..1.04 rows=4 width=10) (actual time=0.013..0.014 rows=4 loops=1)',),
 ('                    Buckets: 1024  Batches: 1  Memory Usage: 9kB',),
 ('                    ->  Seq Scan on dim_src d  (cost=0.00..1.0

### Создаем индексы для таблицы fact_sessions

In [166]:
cur.execute('''
            CREATE INDEX idx_src_id_visit_dttm
            ON fact_sessions(src_id, visit_dttm);
            ''')

### Обновление статистики

In [168]:
cur.execute('''
            analyze fact_sessions
            ''')
cur.execute('''
            analyze dim_src
            ''')

### Смотрим время выполнения запроса с учетом индексов

In [171]:
cur.execute('''
            explain analyze
            select 
            d.src_name,
            count(distinct f.user_id) as count_users,
            count(f.session_id) as count_sessions
            from fact_sessions f
            left join dim_src d
            on d.src_id = f.src_id
            where purchase_flg = 1
            and visit_dttm >= '2022-01-01' 
            group by src_name
            ''')
cur.fetchall()

[('GroupAggregate  (cost=2.36..2.47 rows=4 width=22) (actual time=0.061..0.071 rows=3 loops=1)',),
 ('  Group Key: d.src_name',),
 ('  ->  Sort  (cost=2.36..2.38 rows=7 width=14) (actual time=0.048..0.051 rows=6 loops=1)',),
 ('        Sort Key: d.src_name',),
 ('        Sort Method: quicksort  Memory: 25kB',),
 ('        ->  Hash Left Join  (cost=1.09..2.27 rows=7 width=14) (actual time=0.029..0.035 rows=6 loops=1)',),
 ('              Hash Cond: (f.src_id = d.src_id)',),
 ('              ->  Seq Scan on fact_sessions f  (cost=0.00..1.14 rows=7 width=12) (actual time=0.011..0.014 rows=6 loops=1)',),
 ("                    Filter: ((visit_dttm >= '2022-01-01'::date) AND (purchase_flg = 1))",),
 ('                    Rows Removed by Filter: 3',),
 ('              ->  Hash  (cost=1.04..1.04 rows=4 width=10) (actual time=0.010..0.011 rows=4 loops=1)',),
 ('                    Buckets: 1024  Batches: 1  Memory Usage: 9kB',),
 ('                    ->  Seq Scan on dim_src d  (cost=0.00..1.0

In [ ]:
cur.close()
conn.close()